In [1]:
import import_ipynb
import numpy as np
from scipy import stats
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from data_tools import *
from args import parser
args, unknown = parser.parse_known_args()
tf.random.set_seed(42)

importing Jupyter notebook from data_tools.ipynb
importing Jupyter notebook from args.ipynb


In [2]:
def unet(pretrained_weights = None,input_size = (128,128,1)):
    size_filter_in = 16
    #kernel_init = 'glorot_uniform'
    kernel_init=  tf.keras.initializers.GlorotUniform(seed=42)
    #kernel_init = 'he_normal'
    activation_layer = None 
    inputs = Input(input_size)
    conv1 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(inputs)
    conv1 = LeakyReLU()(conv1)
    conv1 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv1)
    conv1 = LeakyReLU()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool1)
    conv2 = LeakyReLU()(conv2)
    conv2 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv2)
    conv2 = LeakyReLU()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool2)
    conv3 = LeakyReLU()(conv3)
    conv3 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv3)
    conv3 = LeakyReLU()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool3)
    conv4 = LeakyReLU()(conv4)
    conv4 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv4)
    conv4 = LeakyReLU()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool4)
    conv5 = LeakyReLU()(conv5)
    conv5 = Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv5)
    conv5 = LeakyReLU()(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(size_filter_in*8, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(drop5))
    up6 = LeakyReLU()(up6)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge6)
    conv6 = LeakyReLU()(conv6)
    conv6 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv6)
    conv6 = LeakyReLU()(conv6)
    up7 = Conv2D(size_filter_in*4, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv6))
    up7 = LeakyReLU()(up7)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge7)
    conv7 = LeakyReLU()(conv7)
    conv7 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv7)
    conv7 = LeakyReLU()(conv7)
    up8 = Conv2D(size_filter_in*2, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv7))
    up8 = LeakyReLU()(up8)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge8)
    conv8 = LeakyReLU()(conv8)
    conv8 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv8)
    conv8 = LeakyReLU()(conv8)
    
    up9 = Conv2D(size_filter_in, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv8))
    up9 = LeakyReLU()(up9)
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge9)
    conv9 = LeakyReLU()(conv9)
    conv9 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = LeakyReLU()(conv9)
    conv9 = Conv2D(2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = LeakyReLU()(conv9)
    conv10 = Conv2D(1, 1, activation = 'tanh')(conv9)

    model = Model(inputs,conv10)
    
    model.compile(optimizer = 'adam', loss = tf.keras.losses.Huber(), metrics = ['mae'])
    return model

In [3]:
#Path were to read spectrograms of noisy voice and clean voice
path_save_spectrogram = args.path_save_spectrogram
fig_path= args.fig_dir

#path to find pre-trained weights / save models
noise_weigth_path= args.noise_weigths_folder
voice_weigth_path= args.voice_weigths_folder

#pre trained model
name_model = args.name_model

#Training from scratch vs training from pre-trained weights
training_from_scratch = args.training_from_scratch

#epochs for training
epochs = args.epochs

#batch size for training
batch_size = args.batch_size

SNR= 10

In [4]:
epochs, batch_size

(200, 32)

In [ ]:
training_from_scratch = args.training_from_scratch
X_in = np.load(path_save_spectrogram +'noisy_concat= '+str(SNR)+".npy")
X_ou = np.load(path_save_spectrogram +'voice_concat= '+str(SNR)+".npy")

#Model of noise to predict
X_ou = X_in - X_ou

#Check distribution
print(stats.describe(X_in.reshape(-1,1)))
print(stats.describe(X_ou.reshape(-1,1)))

#to scale between -1 and 1
X_in = scaled_in(X_in)
X_ou = scaled_ou(X_ou)

#Check new distribution
print('After Scaling')
print(stats.describe(X_in.reshape(-1,1)))
print(stats.describe(X_ou.reshape(-1,1)))

#Reshape for training
X_in = X_in[:,:,:]
X_in = X_in.reshape(X_in.shape[0],X_in.shape[1],X_in.shape[2],1)

X_ou = X_ou[:,:,:]
X_ou = X_ou.reshape(X_ou.shape[0],X_ou.shape[1],X_ou.shape[2],1)

X_train, X_test, y_train, y_test= train_test_split(X_in, X_ou, test_size=0.10, random_state=42)

#If training from scratch
if training_from_scratch:
    generator_nn= unet()

#If training from pre-trained weights
else:
    generator_nn= unet(pretrained_weights = voice_weigth_path+name_model+'= '+str(SNR)+'.h5')

#Save best models to disk during training
checkpoint = ModelCheckpoint(voice_weigth_path+'/model_unet= '+str(SNR)+'.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

#Training
history = generator_nn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=[checkpoint], verbose=1, validation_data=(X_test, y_test))

#Save json file to the disk
model_json = generator_nn.to_json()
with open(voice_weigth_path+'/' + 'model_unet= '+str(SNR)+'.json', 'w') as json_file:
    json_file.write(model_json)

DescribeResult(nobs=393216000, minmax=(array([-34.76199476+0.j]), array([39.37152901+0.j])), mean=array([0.00047881+0.j]), variance=array([0.2401094]), skewness=array([0.1078933+0.j]), kurtosis=array([57.87406579+0.j]))
DescribeResult(nobs=393216000, minmax=(array([-26.09865901+0.j]), array([23.34109878+0.j])), mean=array([0.00036607+0.j]), variance=array([0.12240186]), skewness=array([0.14808116+0.j]), kurtosis=array([76.66244846+0.j]))
After Scaling
DescribeResult(nobs=393216000, minmax=(array([0.2247601+0.j]), array([1.70743058+0.j])), mean=array([0.92000958+0.j]), variance=array([9.604376e-05]), skewness=array([0.1078933+0.j]), kurtosis=array([57.87406579+0.j]))
DescribeResult(nobs=393216000, minmax=(array([-0.39144706+0.j]), array([0.21147681+0.j])), mean=array([-0.07316627+0.j]), variance=array([1.82037275e-05]), skewness=array([0.14808116+0.j]), kurtosis=array([76.66244846+0.j]))
Train on 21600 samples, validate on 2400 samples
Epoch 1/200
21568/21600 [==========================

In [ ]:
#Plot the data
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize= (10,5))
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.yscale('log')
plt.title('Training and validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
fig.savefig(fig_path+str(SNR)+'.jpg')
fig.savefig(fig_path+str(SNR)+'.pdf')